In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import box

# Read the CSV
df = pd.read_csv("Coordinates_GEE.csv")

# Create polygons from bounding box
df['geometry'] = df.apply(lambda row: box(row['left'], row['bottom'], row['right'], row['top']), axis=1)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Save as shapefile
print(gdf.head())
print(gdf.crs)
print(gdf.total_bounds)
gdf.to_file("bounding_boxes.shp")


        left      right       top    bottom  name  \
0 -63.210069 -63.164075 -0.532219 -0.578213     0   
1 -63.210069 -63.164075 -0.624207 -0.670200     1   
2 -63.210069 -63.164075 -0.670200 -0.716194     2   
3 -63.210069 -63.164075 -0.716194 -0.762188     3   
4 -63.210069 -63.164075 -0.756439 -0.802432     4   

                                            geometry  
0  POLYGON ((-63.16408 -0.57821, -63.16408 -0.532...  
1  POLYGON ((-63.16408 -0.6702, -63.16408 -0.6242...  
2  POLYGON ((-63.16408 -0.71619, -63.16408 -0.670...  
3  POLYGON ((-63.16408 -0.76219, -63.16408 -0.716...  
4  POLYGON ((-63.16408 -0.80243, -63.16408 -0.756...  
EPSG:4326
[-68.94673277  -8.11069501 -47.44382241   1.29563027]


In [5]:
import simplekml
kml = simplekml.Kml()

# Add polygons to KML
for idx, row in df.iterrows():
    coords = [
        (row['left'], row['bottom']),
        (row['left'], row['top']),
        (row['right'], row['top']),
        (row['right'], row['bottom']),
        (row['left'], row['bottom'])  # close the polygon
    ]
    pol = kml.newpolygon(name=f"Box {idx+1}", outerboundaryis=coords)
    # pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.blue)
    
    pol.style.polystyle.color = simplekml.Color.changealphaint(0, simplekml.Color.red)  # fill color
    pol.style.linestyle.color = simplekml.Color.black  # outline color
    pol.style.linestyle.width = 2

# Save KML file
kml.save("bounding_boxes.kml")